In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Semantic Text Similarity

In [8]:
# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial
from typing import Tuple, List
import torch

In [9]:
# Load stopwords in spanish
import nltk
from nltk.corpus import stopwords
#STOPWORDS_ES = {"yo", "tú", "él", "ella", "nosotros", "vosotros", "ellos", "de", "a", }
STOPWORDS_CAT = {"jo", "tu", "ell", "ella", "nosaltres", "vosaltres", "ells", "elles", "de", "a", }
#STOPWORDS_CAT = stopwords.words("catalan")

In [10]:
# Define preprocessing
def preprocess(sentence: str) -> List[str]:
    preprocessed = simple_preprocess(sentence)
    preprocessed = [token for token in preprocessed if token not in STOPWORDS_CAT]
    return preprocessed

# Load Vectors

In [11]:
'''# WORD 2 VEC
WORD_EMBEDDING_FILE = '/content/drive/MyDrive/2n/PLH/PLH-4/cc.ca.300.bin'
# Load with gensim
from gensim.models import fasttext
wv_model = fasttext.load_facebook_vectors(WORD_EMBEDDING_FILE)

#WORD_EMBEDDING_FILE = load_dataset("projecte-aina/catalan_general_crawling")'''

'# WORD 2 VEC\nWORD_EMBEDDING_FILE = \'/content/drive/MyDrive/2n/PLH/PLH-4/cc.ca.300.bin\'\n# Load with gensim\nfrom gensim.models import fasttext\nwv_model = fasttext.load_facebook_vectors(WORD_EMBEDDING_FILE)\n\n#WORD_EMBEDDING_FILE = load_dataset("projecte-aina/catalan_general_crawling")'

In [ ]:
# ROBERTA
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")
model = AutoModelForMaskedLM.from_pretrained("projecte-aina/roberta-base-ca-v2")

import torch
from transformers import RobertaTokenizer, RobertaModel
from sklearn.decomposition import PCA
import numpy as np

# Cargar el modelo y el tokenizer de Roberta
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
#model = RobertaModel.from_pretrained('roberta-base')

# Extraer los embeddings del vocabulario
def get_word_embeddings(model, tokenizer):
    embeddings = model.get_input_embeddings().weight.detach().cpu().numpy()
    vocab = tokenizer.get_vocab()
    word_embeddings = {word: embeddings[idx] for word, idx in vocab.items()}
    return word_embeddings

# Obtener los embeddings
word_embeddings = get_word_embeddings(model, tokenizer)

# Opcional: Reducir la dimensionalidad de 768 a 300
def reduce_dimensionality(word_embeddings, n_components=300):
    words = list(word_embeddings.keys())
    embeddings = np.array(list(word_embeddings.values()))
    pca = PCA(n_components=n_components)
    reduced_embeddings = pca.fit_transform(embeddings)
    reduced_word_embeddings = {word: reduced_embeddings[idx] for idx, word in enumerate(words)}
    return reduced_word_embeddings
reduced_word_embeddings = reduce_dimensionality(word_embeddings, n_components=300)



In [ ]:
'''from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import numpy as np

# Load the Roberta model and tokenizer
roberta_tokenizer = AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")
roberta_model = AutoModelForMaskedLM.from_pretrained("projecte-aina/roberta-base-ca-v2").roberta

def get_roberta_embeddings(roberta_model, roberta_tokenizer):
    vocab = roberta_tokenizer.get_vocab()
    embeddings = {}
    for word, idx in vocab.items():
        inputs = roberta_tokenizer(word, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = roberta_model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings[word] = embedding
    return embeddings

word_embeddings = get_roberta_embeddings(roberta_model, roberta_tokenizer)

from sklearn.decomposition import PCA

def reduce_dimensionality(word_embeddings, n_components=300):
    words = list(word_embeddings.keys())
    embeddings = np.array(list(word_embeddings.values()))
    pca = PCA(n_components=n_components)
    reduced_embeddings = pca.fit_transform(embeddings)
    reduced_word_embeddings = {word: reduced_embeddings[idx] for idx, word in enumerate(words)}
    return reduced_word_embeddings

reduced_word_embeddings = reduce_dimensionality(word_embeddings, n_components=300)
'''

In [ ]:
class CustomKeyedVectors:
    def __init__(self, word_embeddings):
        self.word_embeddings = word_embeddings

    def __getitem__(self, word):
        return self.word_embeddings[word]

    def __contains__(self, word):
        return word in self.word_embeddings

# Crear el objeto wv_model
wv_model = CustomKeyedVectors(reduced_word_embeddings)


In [ ]:
'''# SPACY
!python -m spacy download ca_core_news_md
import spacy
model = spacy.load("ca_core_news_md")
wv_model = model.vocab.vectors
'''

In [ ]:
# ONE HOT ENCODING

In [ ]:
# If you want, you can use mmaps
USE_MMAP = False
if USE_MMAP:
    from gensim.models.fasttext import FastTextKeyedVectors
    MMAP_PATH = 'cc.ca.gensim.bin'
    # wv_model.save(MMAP_PATH)
    wv_model = FastTextKeyedVectors.load(MMAP_PATH, mmap='r')

In [ ]:
# Sample data
input_pairs_example = [
    ('Me gusta el futbol', 'Disfruto viendo partidos de futbol', 4),
    ('El cielo está despejado', 'Hace un día bonito', 4.5),
    ('Me encanta viajar', 'Explorar nuevos lugares es una pasión', 3.5),
    ('Prefiero el verano', 'No me gusta el frío del invierno', 2.5),
    ('Tengo hambre', '¿Qué hay para cenar?', 2),
    ('La música me relaja', 'Escuchar música es una terapia', 3),
    ('El libro es emocionante', 'No puedo dejar de leerlo', 4),
    ('Me gusta la pizza', 'Es mi comida favorita', 4.5),
    ('Estoy cansado', 'Necesito hacer una siesta', 1.5),
    ('Hoy hace mucho calor', 'Es un día sofocante', 3.5)
]

In [ ]:
# Real data
!pip install datasets
from datasets import load_dataset
#dataset = load_dataset("PlanTL-GOB-ES/sts-es")
dataset = load_dataset("projecte-aina/sts-ca")

In [ ]:
'''dataset['train'] = dataset['train'].select(range(len(dataset['train']) // 200))
dataset['validation'] = dataset['validation'].select(range(len(dataset['validation']) // 200))
dataset['test'] = dataset['test'].select(range(len(dataset['test']) // 200))

'''

In [ ]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["test"].to_list()]

In [ ]:
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [simple_preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)

In [ ]:
print(sentence_pairs[0])

In [ ]:
# Cálculo de los pesos TF-IDF para las oraciones pre-procesadas
corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
modelo_tfidf = TfidfModel(corpus)

In [ ]:
# Per word 2 vec
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc)
    tf_idf = tf_idf_model[bow]
    vectors, weights = [], []
    for word_index, weight in tf_idf:
        word = dictionary.get(word_index)
        if word in wv_model:
            vectors.append(wv_model[word])
            weights.append(weight)
    return vectors, weights

def map_pairs_vells(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        tf_idf_model: TfidfModel = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess(sentence_1)
        sentence_2_preproc = preprocess(sentence_2)
        # Si usamos TF-IDF
        if tf_idf_model is not None:
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vector1 = np.average(vectors1, weights=weights1, axis=0, )
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        else:
            # Cálculo del promedio de los word embeddings
            vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]
            vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
            vector1 = np.mean(vectors1, axis=0)
            vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [ ]:
def map_pairs(sentence_pairs, dictionary=None, tf_idf_model=None):
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess(sentence_1)
        sentence_2_preproc = preprocess(sentence_2)
        if tf_idf_model is not None:
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model)
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model)
            if weights1 and weights2 and sum(weights1) != 0 and sum(weights2) != 0:
                vector1 = np.average(vectors1, weights=weights1, axis=0)
                vector2 = np.average(vectors2, weights=weights2, axis=0)
            else:
                vector1 = np.mean(vectors1, axis=0) if vectors1 else np.zeros(300)
                vector2 = np.mean(vectors2, axis=0) if vectors2 else np.zeros(300)
        else:
            vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]
            vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
            vector1 = np.mean(vectors1, axis=0) if vectors1 else np.zeros(300)
            vector2 = np.mean(vectors2, axis=0) if vectors2 else np.zeros(300)
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores



In [ ]:
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped = map_pairs(input_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped_train = map_pairs(input_pairs,  tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_val = map_pairs(input_pairs_val, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_test = map_pairs(input_pairs_test, tf_idf_model=modelo_tfidf, dictionary=diccionario, )

In [ ]:
for vectors, similitud in mapped[:5]:
    print(f"Pares de vectores: {vectors[0].shape}, {vectors[1].shape}")
    print(f"Puntuación de similitud: {similitud}")

In [ ]:
# Define the Model
import tensorflow as tf

def build_and_compile_model(embedding_size: int = 300, learning_rate: float = 1e-3) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Hidden layer
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 = first_projection(input_1)
    projected_2 = first_projection(input_2)

    # Compute the cosine distance using a Lambda layer
    def cosine_distance(x):
        x1, x2 = x
        x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
        x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
        return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))

    output = tf.keras.layers.Lambda(cosine_distance)([projected_1, projected_2])
    # Define output
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adamax(learning_rate))
    return model

In [ ]:
def build_and_compile_model_better(embedding_size: int = 300, learning_rate: float = 1e-3) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Hidden layer
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 =  first_projection(input_1)
    projected_2 = first_projection(input_2)

    # Compute the cosine distance using a Lambda layer
    def normalized_product(x):
        x1, x2 = x
        x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
        x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
        return x1_normalized * x2_normalized

    output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
    output = tf.keras.layers.Dropout(0.1)(output)
    output = tf.keras.layers.Dense(
        16,
        activation="relu",
    )(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(
        1,
        activation="sigmoid",
    )(output)

    output = tf.keras.layers.Lambda(lambda x: x * 5)(output)

    # Define output
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))
    return model

In [ ]:
# Define training constants
batch_size: int = 64
num_epochs: int = 64

In [ ]:
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, )

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped_train)
x_val, y_val = pair_list_to_x_y(mapped_val)

In [ ]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
# Show shapes
x_train[0].shape, x_train[1].shape, y_train.shape

In [ ]:
# Construir y compilar el modelo
model = build_and_compile_model()
#tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True, )
print(model.summary())

In [ ]:
# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

In [ ]:
from scipy.stats import pearsonr
x_test, y_test = pair_list_to_x_y(mapped_test)

In [ ]:
# Baseline
def compute_pearson_baseline(x_, y_):
    y_pred_baseline = []
    for v1, v2 in zip(*x_):
        d = 1.0 - spatial.distance.cosine(v1, v2)
        y_pred_baseline.append(d)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred_baseline, y_.flatten())
    return correlation
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (baseline-train): {compute_pearson_baseline(x_train, y_train)}")
print(f"Correlación de Pearson (baseline-validation): {compute_pearson_baseline(x_val, y_val)}")
print(f"Correlación de Pearson (baseline-test): {compute_pearson_baseline(x_test, y_test)}")
with open(f'/content/drive/MyDrive/2n/PLH/PLH-4/resultats_{batch_size}_{num_epochs}_baseline.txt', 'w', encoding='utf-8') as file_opened:
  file_opened.write('Correlación de Pearson (baseline-train):\n')
  file_opened.write(str(compute_pearson_baseline(x_train, y_train)) + '\n')
  file_opened.write('Correlación de Pearson (baseline-validation):\n')
  file_opened.write(str(compute_pearson_baseline(x_val, y_val)) + '\n')
  file_opened.write('Correlación de Pearson (baseline-test):\n')
  file_opened.write(str(compute_pearson_baseline(x_test, y_test)) + '\n')

In [ ]:
def compute_pearson(x_, y_):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    y_pred = model.predict(x_)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train)}")
print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val)}")
print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test)}")
with open(f'/content/drive/MyDrive/2n/PLH/PLH-4/resultats_{batch_size}_{num_epochs}.txt', 'w', encoding='utf-8') as file_opened:
  file_opened.write('Correlación de Pearson (train):\n')
  file_opened.write(str(compute_pearson(x_train, y_train)) + '\n')
  file_opened.write('Correlación de Pearson (validation):\n')
  file_opened.write(str(compute_pearson(x_val, y_val)) + '\n')
  file_opened.write('Correlación de Pearson (test):\n')
  file_opened.write(str(compute_pearson(x_test, y_test)) + '\n')